# Welcome to pystardog

Press the Restart & Run All button to run all the cells in this notebook and view the output.

In [ ]:
import io
import stardog
import pandas as pd
import seaborn as sns
from configparser import ConfigParser

In [ ]:
# Get credentials from file
# file contains DEFAULT section plus override sections
config_section = 'doghouse'
parser = ConfigParser()
_ = parser.read('../CREDENTIALS.config')

url = parser.get(config_section, 'url')
user = parser.get(config_section, 'user')
password = parser.get(config_section, 'password')
db = 'insurance'
api_endpoint = 'query'

## Specify Stardog connection details

In [ ]:
connection_details = {
    'endpoint': url,
    'username': user,
    'password': password
}


## Create a new database in Stardog

Drop the database if it already exists.

In [ ]:
database_name = 'insurance'

## Connect to the Stardog database

In [ ]:
conn = stardog.Connection(database_name, **connection_details)

## Load the sample data
### Start a transaction

In [ ]:
conn.begin()

## Query the database

This query returns the crime stats for Washington DC by crime type and zip code.

In [ ]:
query = """
PREFIX sqs: <tag:stardog:api:sqs:>
PREFIX : <http://api.stardog.com/>

select * { ?statIRI a :Crime_Stats;
                   :Crime_Type ?offense;
                   :Crime_Count ?crimeCount;
                   :Crime_Zip ?zipCode;
                   :Occurred_In ?zipCodeIri.
          ?zipCodeIri a :Zip_Codes.
          } 
"""

csv_results = conn.select(query, content_type='text/csv')
df = pd.read_csv(io.BytesIO(csv_results))

## Preview Crime Data

In [ ]:
df.head()

## Plot total crime stats in Washington DC

In [ ]:
#Plotting a bar chart
import matplotlib.pyplot as plt
plt.figure(figsize=[5,4])
#df['offense'].value_counts().plot.barh()
df.groupby('offense').crimeCount.sum().sort_values(ascending=False).plot(kind="bar")
sns.set(style="darkgrid")
plt.show()

In [ ]:
## Plot total crime stats in Washington DC by Zip Code

In [ ]:
#Plotting a bar chart
import matplotlib.pyplot as plt
plt.figure(figsize=[6,5])
#df['offense'].value_counts().plot.barh()
df.plot.scatter(x='zipCode',
                      y='offense')
sns.set(style="darkgrid")
plt.show()

### Clean up the connection

Normally you would use a `with statement` similar to line 3.

In [ ]:
conn.__exit__()